In [3]:
!pip install selenium openpyxl

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import csv
import datetime
import time
import os
from selenium import webdriver

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np


In [2]:
def set_driver(url):
    
    ser = Service(DRIVER_PATH)
    options = webdriver.ChromeOptions()  # Set up Chinese
    # options.add_argument('--headless')
    options.add_argument('lang=zh_CN.UTF-8')  # Replacement of head
    options.add_argument(
        'User-Agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"')
    options.add_argument("--disable-infobars")
    options.add_argument("--headless")
    # PROXY = "11.456.448.110:8080"
    # options.add_argument('--proxy-server=%s' % PROXY)
    driver = webdriver.Chrome(
        service=ser,
        options=options)

    driver.get(url)
    print("finish setup driver")
    return driver

    
def init_filename(source, subject):
    # /呱吉/PTT.csv
    # /呱吉/FB.csv
    # ...
    folder = f"/{source}/"
    return subject + folder


def write_file():
    out_filename = ""
    with open(out_filename, 'a') as out_file:
        for line in in_file:

            out_file.write(parsed_line)


def init_csv(file_name):

    with open(file_name, 'w+') as out_file:
        write = csv.writer(out_file)
        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        # PTT, 呱吉吃大便, http://, '2021-11-17', XXXX, XXXXX, 23
        write.writerow(header)


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')

def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "": pass
            if row[1] == "頻道(官方帳號)": continue
            ytr_name = []
            ytr_name.append(row[1].split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(set(ytr_name)))
    return ytr_list

## constant

In [3]:
DRIVER_PATH = '/home/bobo/Desktop/bobo/ML_final/bobo/crawlers/chromedriver'
keywords_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=館長"
tag_url = "https://www.ettoday.net/news/tag/統神/"

In [4]:
ytr_list = ["谷阿莫", "黃氏兄弟", "聖結石", "曾聖傑"]


## tag

In [5]:
def scroll_to_button(driver):
    reached_page_end = False
    last_height = driver.execute_script("return document.body.scrollHeight")

    while not reached_page_end:
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True
        else:
            last_height = new_height
    print("reach end")
    # driver.quit()
    
def crawl_page_tag(driver, filename):
    block_div = driver.find_element(By.CLASS_NAME, "c1")
    title_list = block_div.find_elements(By.XPATH,
        ".//div[@class='piece clearfix']")
    
    if title_list == []:
        print(title_list)
        print("is empty")
        return

    press = title_list[-1].find_element(By.TAG_NAME,
        "h3").find_element(By.TAG_NAME,"a")
    press.send_keys(Keys.NULL)

    print("len: ", len(title_list))

    filename = filename
    with open(filename, 'a') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for i in title_list:
            dic['Source'] = "ettoday"
            dic['Title'] = i.find_element(By.TAG_NAME,'h3').text.replace("  ", " ")
            dic['Link'] = i.find_element(By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = i.find_element(By.CLASS_NAME,'date').text
            dic['Summary'] = i.find_element(By.CLASS_NAME,'summary').text
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)

In [7]:
# tag search
def crawl_tag(path, tag_url):
    driver = set_driver(tag_url)

    scroll_to_button(driver)

    time.sleep(10)

    crawl_page_tag(driver, filename=path+"ettoday_tag.csv")

    driver.quit()

## keywords

In [8]:
def crawl_page_keyword(driver, filename):
    block_div = driver.find_element(By.ID, "primary")
    box_list = block_div.find_elements(By.CLASS_NAME, "box_2")
    
    if box_list == []:
        print("is empty")
        return


    filename = filename
    with open(filename, 'a') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for box in box_list:
            dic['Source'] = "ettoday"
            dic['Title'] = box.find_element(By.TAG_NAME, "h2").text.replace("\u3000", " ")
            dic['Link'] = box.find_element(By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = box.find_element(By.CLASS_NAME, 'date').text.split("/ ")[1].replace(")", "")
            dic['Summary'] = box.find_element(By.CLASS_NAME, 'detail').text
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)


def check_exists_by_class(driver, class_name):
    try:
        driver.find_element(By.CLASS_NAME, class_name)
    except NoSuchElementException:
        return False
    return True

def get_max_page(driver):
    info = driver.find_element(By.CLASS_NAME, "info").text
    for t in ["第", "頁", "共", "頁"]:
        info = info.replace(t, "")
    info = info.replace(" ", "")
    current_page, max_page = map(int, info.split("|"))
    current_page, max_page
    return max_page


In [9]:
# search by keywords
def crawl_keyword(path, keywords_url):
    driver = set_driver(keywords_url)

    for i in range(get_max_page(driver)):
        print("page ", i)
        crawl_page_keyword(driver, filename=path+"ettoday_keywords.csv")
        time.sleep(5)
        next_page = driver.find_element(By.XPATH,"//*[text()=' > ']")
        next_page.click()

    driver.quit()

## ytr crawling

In [10]:
ytr_list = generate_ytr_name_list("Youtuber名單 - 道歉.csv")

In [ ]:
base_path = "data/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for ytr_name in ytr_list:
    print(ytr_name)
    folder_path = base_path + ytr_name[0] + "/"
    
    # if folder exists, next ytr
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        continue
    
    
    init_csv(folder_path+"ettoday_tag.csv")
    init_csv(folder_path+"ettoday_keywords.csv")
    
    
    for name in ytr_name:
        keywords_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=" + name
        tag_url = "https://www.ettoday.net/news/tag/" + name + "/"
    
        crawl_tag(folder_path, tag_url)
        crawl_keyword(folder_path, keywords_url)
        
    print("*"*20)
    print()

['芋圓柚子OEUR.studio', '阿神']
['阿滴', '阿滴英文', '都省瑞']
finish setup driver
reach end
len:  186
finish setup driver
page  0
page  1
